In [1]:
import ipywidgets as widgets
from IPython.display import display
from matplotlib import pyplot as plt
import numpy as np

from zest.circuit import Circuit, SubCircuitDef, SubCircuitInst
from zest.components import Capacitor, PulsedVoltageSource, Terminal, VoltageSource, Resistor
from lif_neuron_clean import create_voltage_controlled_switch_with_hysteresis_subcircuit


In [2]:
# Copy over the subcircuit definitions
def create_inverter_subcircuit(v_high=1.0, v_low=0.0, v_th=0.5):
    inverter_def = SubCircuitDef("inverter")

    pin_in = Terminal()
    pin_out = Terminal()

    inverter_def.add_pin("IN", pin_in)
    inverter_def.add_pin("OUT", pin_out)

    expression = f"V = {v_high} - (V(IN) > {v_th}) * ({v_high} - {v_low})"
    inverter_def.include_model(f"BVOUT OUT 0 {expression}")

    return inverter_def

def create_switched_cap_subcircuit(c=1e-6, switch_on_voltage=0.5):
    switched_cap_def = SubCircuitDef("switched_cap")

    pin_in = Terminal()
    pin_out = Terminal()
    pin_switch = Terminal()

    switched_cap_def.add_pin("IN", pin_in)
    switched_cap_def.add_pin("OUT", pin_out)
    switched_cap_def.add_pin("SW", pin_switch)

    switch_def = create_voltage_controlled_switch_with_hysteresis_subcircuit(on_voltage=switch_on_voltage, hysteresis_voltage=0)
    sw1 = SubCircuitInst(switch_def)
    sw2 = SubCircuitInst(switch_def)
    inverter_def = create_inverter_subcircuit(v_high=1.0, v_low=0.0)
    inverter = SubCircuitInst(inverter_def)
    c1 = Capacitor(c)

    switched_cap_def.add_component(sw1)
    switched_cap_def.add_component(sw2)
    switched_cap_def.add_component(c1)

    switched_cap_def.wire(sw1.N1, pin_in)
    switched_cap_def.wire(sw1.N2, c1.pos)
    switched_cap_def.wire(sw2.N1, c1.pos)
    switched_cap_def.wire(c1.neg, switched_cap_def.gnd)
    switched_cap_def.wire(sw2.N2, pin_out)

    switched_cap_def.wire(sw1.NCP, pin_switch)
    switched_cap_def.wire(pin_switch, inverter.IN)
    switched_cap_def.wire(inverter.OUT, sw2.NCP)
    switched_cap_def.wire(sw1.NCM, switched_cap_def.gnd)
    switched_cap_def.wire(sw2.NCM, switched_cap_def.gnd)

    return switched_cap_def


In [3]:
def simulate_switched_cap(f_switch, c_switch, c_out):
    """Run simulation with given parameters and return results"""
    switched_cap_def = create_switched_cap_subcircuit(c=c_switch, switch_on_voltage=0.5)
    switched_cap = SubCircuitInst(switched_cap_def)

    circuit = Circuit("switched_cap_test")
    circuit.add_component(switched_cap)

    # Clock signal for switching
    period = 1/f_switch
    v_switch = PulsedVoltageSource(
        v1=0.0,
        v2=1.0,
        td=0.0,
        tr=1e-9,
        tf=1e-9,
        pw=period/2,  # 50% duty cycle
        per=period
    )
    circuit.add_component(v_switch)
    circuit.wire(v_switch.neg, circuit.gnd)

    # Input voltage - step input
    v_in = PulsedVoltageSource(
        v1=0.0,
        v2=1.0,
        td=1e-3,    # Start after 1ms
        tr=1e-9,
        tf=1e-9,
        pw=10e-3,   # 10ms pulse
        per=20e-3   # 20ms period
    )
    circuit.add_component(v_in)
    circuit.wire(v_in.neg, circuit.gnd)

    # Add load resistor and output capacitor
    r_load = Resistor(1e3)  # 1kΩ load resistor
    c_out_comp = Capacitor(c_out)  # Output capacitor
    circuit.add_component(r_load)
    circuit.add_component(c_out_comp)
    
    circuit.wire(v_switch.pos, switched_cap.SW)
    circuit.wire(v_in.pos, switched_cap.IN)
    circuit.wire(switched_cap.OUT, r_load.n1)
    circuit.wire(r_load.n2, circuit.gnd)
    
    circuit.wire(switched_cap.OUT, c_out_comp.pos)
    circuit.wire(c_out_comp.neg, circuit.gnd)
    
    circuit.wire(v_in.neg, circuit.gnd)

    # Simulate
    results = circuit.simulate_transient(end_time=20e-3, step_time=1e-5)
    
    return results

def update_plot(f_switch, c_switch_exp, c_out_exp):
    """Update plot with new parameters"""
    # Convert exponential sliders to actual values
    c_switch = 10**c_switch_exp
    c_out = 10**c_out_exp
    
    # Run simulation
    results = simulate_switched_cap(f_switch, c_switch, c_out)
    
    # Calculate theoretical values
    r_eq = 1/(f_switch * c_switch)
    tau_theoretical = r_eq * c_out
    
    # Get results
    time = results.get_time_vector()
    v_in = results.get_node_voltage('v_in.pos')
    v_out = results.get_node_voltage('switched_cap.OUT')
    
    # Clear and create new plot
    plt.clf()
    plt.figure(figsize=(12, 8))
    
    # Plot results
    plt.subplot(2, 1, 1)
    plt.plot(time*1e3, v_in, 'r-', linewidth=2, label='Input Voltage')
    plt.plot(time*1e3, v_out, 'b-', linewidth=2, label='Output Voltage')
    plt.xlabel('Time (ms)')
    plt.ylabel('Voltage (V)')
    plt.title('Switched Capacitor Response')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Add text with calculations
    plt.subplot(2, 1, 2)
    plt.axis('off')
    text = f"Switching Frequency: {f_switch/1e3:.1f} kHz\n"
    text += f"Switching Capacitor: {c_switch*1e6:.1f} µF\n"
    text += f"Output Capacitor: {c_out*1e6:.1f} µF\n"
    text += f"Equivalent Resistance: {r_eq/1e3:.2f} kΩ\n"
    text += f"Theoretical Time Constant: {tau_theoretical*1e3:.2f} ms"
    plt.text(0.1, 0.5, text, fontsize=12, family='monospace')
    
    plt.tight_layout()
    plt.show()


In [4]:
# Create interactive widgets
f_switch_slider = widgets.FloatSlider(
    value=1000,
    min=100,
    max=10000,
    step=100,
    description='Freq (Hz):',
    style={'description_width': 'initial'}
)

c_switch_slider = widgets.FloatSlider(
    value=-6,  # 10^-6 = 1µF
    min=-9,    # 1nF
    max=-3,    # 1mF
    step=0.5,
    description='Switch Cap (10^x F):',
    style={'description_width': 'initial'}
)

c_out_slider = widgets.FloatSlider(
    value=-5,  # 10^-5 = 10µF
    min=-9,    # 1nF
    max=-3,    # 1mF
    step=0.5,
    description='Out Cap (10^x F):',
    style={'description_width': 'initial'}
)

# Create interactive output
interactive_plot = widgets.interactive(
    update_plot,
    f_switch=f_switch_slider,
    c_switch_exp=c_switch_slider,
    c_out_exp=c_out_slider
)

# Display the interactive plot
display(interactive_plot)


interactive(children=(FloatSlider(value=1000.0, description='Freq (Hz):', max=10000.0, min=100.0, step=100.0, …